In [31]:
import tensorflow as tf

# Load your Keras model
# Update the path to where the model file is actually stored.
# If stored in Google Drive, mount the drive and provide the full path.
# If downloaded, make sure it's in the correct directory (e.g., /content/).
model_path = "/content/model_2024_hairstyle.keras"  # Replace with the correct path
try:
    model = tf.keras.models.load_model(model_path)
except FileNotFoundError:
    print(f"Error: Model file not found at {model_path}")
    # Handle the error, for example, by downloading the model or specifying a different path
    # ...
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    # Handle other potential errors during model loading

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open("model_2024_hairstyle.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp2ql05k6x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135397370759824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135397369746304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135397369747360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135397370085616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135397370095120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135397370091952: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [32]:
import tensorflow as tf

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="/content/model_2024_hairstyle.tflite")
interpreter.allocate_tensors()

# Get the output details
output_details = interpreter.get_output_details()

# Print the output index
print("Output Index:", output_details[0]['index'])


Output Index: 13


In [3]:
!pip install pillow

In [33]:
import tensorflow as tf

model = tf.keras.models.load_model('/model_2024_hairstyle.keras')
input_shape = model.input_shape  # Example: (None, 224, 224, 3)
print(f"Model expects input size: {input_shape[1:3]}")


Model expects input size: (200, 200)


In [34]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img




In [35]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

# Download and prepare the image
img = download_image(url)
prepared_img = prepare_image(img, target_size=(200, 200))

# Save or use the resized image
prepared_img.save("resized_image.jpg")  # Optional, for verification

In [36]:
import numpy as np

img_array = np.array(prepared_img)
img_array = img_array / 255.0
input_array = np.expand_dims(img_array, axis=0)
first_pixel_r = img_array[0, 0, 0]
print(f"First pixel R channel value: {first_pixel_r}")

First pixel R channel value: 0.23921568627450981


In [37]:
import tensorflow as tf
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image

# Define functions
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# Download and preprocess image
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
prepared_img = prepare_image(img, target_size=(200, 200))

# Convert to NumPy array and preprocess
img_array = np.array(prepared_img) / 255.0  # Scale pixel values
input_data = np.expand_dims(img_array, axis=0).astype(np.float32)  # Add batch dimension

# Load TFLite model
tflite_model_path = "/model_2024_hairstyle.tflite"  # Replace with your model path
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensor indices
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

# Set input data
interpreter.set_tensor(input_index, input_data)

# Run inference
interpreter.invoke()

# Get output
output_data = interpreter.get_tensor(output_index)
print("Model Output:", output_data)


Model Output: [[0.8937741]]


In [38]:
!apt-get update
!apt-get install docker.io


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [39]:
!systemctl start docker


System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down


In [40]:
!docker --version


Docker version 24.0.7, build 24.0.7-0ubuntu2~22.04.1


In [41]:
from google.colab import output
import subprocess

subprocess.run(['docker', 'pull', 'agrigorev/model-2024-hairstyle:v3'])



CompletedProcess(args=['docker', 'pull', 'agrigorev/model-2024-hairstyle:v3'], returncode=1)

In [42]:
subprocess.run(['docker', 'image', 'inspect', 'agrigorev/model-2024-hairstyle:v3', '--format={{.Size}}'])


CompletedProcess(args=['docker', 'image', 'inspect', 'agrigorev/model-2024-hairstyle:v3', '--format={{.Size}}'], returncode=1)

In [43]:
!apt-get update
!apt-get install -y python3-pip


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [44]:
!pip install udocker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.6/119.6 kB 6.8 MB/s eta 0:00:00


In [47]:
!pip install --user udocker




In [48]:
!udocker --version


Error: do not run as root !


# New Section